# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /home/ian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [33]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

import nltk
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report


In [23]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table(table_name = 'InsertTableName', con=engine)

X = df.message
Y = df[['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity',
'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report', ]]

In [4]:
df

,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,a lot and we don't have any news of him. We ha...,"anpil E nou pa gen oken nouvel li, menm ke a n...",direct,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Since last night i cannot get through ( on cel...,De pi ye swa m pa ka pase. Yo toujou di mwen r...,direct,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,you have been said that you ganna put 100 gour...,Nou te di n'ap mete 100 gdes sou telefon nan p...,direct,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,How many aftershocks we can have.,Konbyen sekous ki rete nou ank.,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Haiti honey you're the rose thorns find the fi...,Haiti cherie tu es la rose qui se trouves au c...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Caution to take about the cyclone in Haiti spe...,Mesure de prcaussion prendre dans les sircons...,direct,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
6,Call me when you get this message.,Call me when you get this message.,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,information about season's cyclon.,Information sur la saison cyclone,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,Digiceel! I'd like to know if will have seime ...,DIGICEL MWEN BEZWEN KONNEN ESKE AP GENYEN SEIS...,direct,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"Zb rmd : dl c.lve dl 2, dl rep, dl chase.","Zb rmd : dl c.lve dl 2, dl rep, dl chase",direct,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [8]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
rf = RandomForestClassifier()

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator = rf)),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

0        a lot and we don't have any news of him. We ha...
1        Since last night i cannot get through ( on cel...
2        you have been said that you ganna put 100 gour...
3                       How many aftershocks we can have. 
4        Haiti honey you're the rose thorns find the fi...
5        Caution to take about the cyclone in Haiti spe...
6                     Call me when you get this message.  
7                      information about season's cyclon. 
8        Digiceel! I'd like to know if will have seime ...
9               Zb rmd : dl c.lve dl 2, dl rep, dl chase. 
10       We have a baby that died and another that has ...
11       NOTES: Do not care about this message because ...
12       Good morning i need some information i never g...
13       May i know if the schools will find help for t...
14       Hallo president if the requested i ask you can...
15       NOTES: It's not necessary to translate this me...
16       NOTES: Do not care about this message because .

In [26]:
# perform train test split
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# train classifier
pipeline.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [34]:
# evaluate all steps on test set
y_pred = pipeline.predict(X_test)

for col in y_pred:
    print(classification_report(y_test, y_pred))

ValueError: Unknown label type: (       related  request  offer  aid_related  medical_help  medical_products  \
1512         1        0      0            0             0                 0   
24905        0        0      0            0             0                 0   
14366        0        0      0            0             0                 0   
18482        0        0      0            0             0                 0   
20462        1        0      0            0             0                 0   
11565        1        0      0            1             1                 1   
17870        1        0      0            0             0                 0   
11779        0        0      0            0             0                 0   
526          1        1      0            1             0                 0   
9985         0        0      0            0             0                 0   
11057        1        1      0            1             0                 0   
19812        1     

In [30]:
def display_results(y_test, y_pred): #was:(cv, y_test, y_pred)
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    #print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)
    #print("\nBest Parameters:", cv.best_params_)

In [31]:
# evaluate all steps on test set
y_pred = pipeline.predict(X_test)

display_results(y_test, y_pred)

ValueError: multiclass-multioutput is not supported

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.